In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization,Conv2D,MaxPool2D,Dropout,MaxPooling2D, Convolution2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category= FutureWarning)
%matplotlib inline



from os import listdir
from os.path import isfile, join
import numpy
import cv2
import numpy as np
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.applications.vgg16 import preprocess_input
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from sklearn.metrics import classification_report,confusion_matrix

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, Dropout, BatchNormalization
from keras.models import Model 
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import regularizers
from keras import optimizers
from keras import backend as K 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

import keras.backend as K
from keras.models     import Sequential
from keras.layers     import Dense, Dropout, GlobalMaxPooling2D
from keras.optimizers import Adam, SGD
from keras.applications import MobileNetV2
from keras.callbacks    import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path = '../input/dataset/Data/Train'
test_path = '../input/dataset/Data/Dev'


In [ ]:
train_datagen = ImageDataGenerator( preprocessing_function= tf.keras.applications.vgg16.preprocess_input,
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.25)



test_datagen = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input,rescale=1 / 255.0)

In [ ]:
batch_size = 64
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(48, 48),
    subset = 'training',
    batch_size=batch_size,
    class_mode = 'categorical',
    classes=['happy','sad', 'angry','fear', 'surprise', 'neutral', 'disgust'],
)
valid_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(48, 48),
    batch_size=batch_size,
    subset = 'validation',
    class_mode = 'categorical',
    classes=['happy','sad', 'angry','fear', 'surprise', 'neutral', 'disgust'],
)
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(48, 48),
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

In [ ]:
def prepare_model():
    model = Sequential()
    model.add(Conv2D(256,kernel_size=(3,3),activation='relu',input_shape=(48, 48, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(48, 48, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(48, 48, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(7, activation='softmax'))
    model.compile(loss="categorical_crossentropy",optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
    return model

In [ ]:
model= prepare_model()
model.fit(train_generator,
                    validation_data = valid_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=5)



In [ ]:
model.fit(train_generator,
                    validation_data = valid_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=20)



In [ ]:
model.fit(train_generator,
                    validation_data = valid_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    validation_steps = valid_generator.n//valid_generator.batch_size,
                    epochs=50)


In [ ]:
score = model.evaluate_generator(valid_generator)
print('Test loss:', score[0]*100)
print('Test accuracy:', score[1]*100)

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=test_generator.n/test_generator.batch_size,
verbose=1)

In [ ]:
pred_bool = (pred >0.5)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"file":filenames,
                      "emotion_class":predictions})
results.to_csv("resultsyamosahel%.csv",index=False)